# GPT를 활용한 매장 정보 챗봇

In [2]:
import datetime as dt, pandas as pd, numpy as np, datetime as dt
import os, time , glob, shutil, pymysql, re, requests, pickle, os.path, base64, email, hashlib, crypto, random, sys, openai, tiktoken
from openpyxl import Workbook, load_workbook
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

def ts(num : int) :
    time.sleep(num)
def live_db_conn() :
    conn = pymysql.connect(host='호스트', user='사용자', password='비밀번호',autocommit=True,cursorclass=pymysql.cursors.DictCursor, db = "db")
    return conn


MAX_SECTION_LEN = 1000
SEPARATOR = "\n* "
ENCODING = "cl100k_base"
EMBEDDING_MODEL = "text-embedding-ada-002"
COMPLETIONS_MODEL = "text-davinci-003"
COMPLETIONS_API_PARAMS = {
    "temperature": 0.0,
    "max_tokens": 300,
    "model": COMPLETIONS_MODEL
}


## 데이터 전처리

In [3]:
# 데이터 Select
conn = live_db_conn()

query = """SELECT DISTINCT S.id, ST.name, S.telephone, ST.address, ST.address_road, ST.holiday, ST.optime, ST.subway, ST.etc, ST.description
, S.seat1 table_chair, S.seat2 floor_table_chair, S.room, S.product_price_min , S.product_price_max
, SCT.name store_category_name, SLT.name store_label_name
, CASE WHEN RWG.id IS NOT NULL THEN "Y" ELSE "N" END google_reservation
, S.`option`
, PT.name product_name, P.id product_id, P.is_package, P.is_main, P.is_reserve, P.minimum_buy_count, P.price, PT.description product_description, PT.includes
FROM store S
LEFT JOIN product P ON P.store_id = S.id
LEFT JOIN product_translate PT ON PT.product_id = P.id
LEFT JOIN product_tag PTA ON PTA.product_id = P.id
LEFT JOIN product_tag_master PTM ON PTM.id = PTA.product_tag_master_id
LEFT JOIN product_tag_translate PTT ON PTT.product_tag_master_id = PTM.id
LEFT JOIN store_translate ST ON ST.store_id = S.id
LEFT JOIN store_category SC ON SC.store_id = S.id
LEFT JOIN store_category_master SCM ON SCM.id = SC.store_category_master_id
LEFT JOIN store_category_translate SCT ON SCT.store_category_master_id = SCM.id
LEFT JOIN store_label SL ON SL.store_id = S.id
LEFT JOIN store_label_master SLM ON SLM.id = SL.store_label_master_id
LEFT JOIN store_label_translate SLT ON SLT.store_label_master_id = SLM.id
LEFT JOIN store_award SA ON SA.store_id = S.id
LEFT JOIN store_award_translate_old SAT ON SAT.store_award_id = SA.id
LEFT JOIN store_media SM ON SM.store_id = S.id
LEFT JOIN store_media_translate SMT ON SMT.store_media_id = SM.id
LEFT JOIN reserve_with_google RWG ON RWG.store_id = S.id
LEFT JOIN store_business_time SBT ON SBT.store_id = S.id
WHERE S.id = 10083 AND P.`status` = 'normal' AND P.sale_status = 'sale' AND ST.lang = 'en' AND PT.lang ='en' AND SCT.lang = 'en' AND SLT.lang = 'en' AND PTT.lang = 'en'"""
df = pd.read_sql(query, conn)
conn.close()

In [5]:
# 옵션 데이터 변경
result = []
for i in df["option"].tolist():
    result.append('{0:017b}'.format(i))
df["option_2"] = result

result = []
for i in range(len(df)):
    possible_option = []
    if df.loc[i, "option_2"][16] == "1":
        possible_option.append("Kid-free Zone")
    if df.loc[i, "option_2"][15] == "1":
        possible_option.append("Self service")
    if df.loc[i, "option_2"][14] == "1":
        possible_option.append("High chair")
    if df.loc[i, "option_2"][13] == "1":
        possible_option.append("Delivery")
    if df.loc[i, "option_2"][12] == "1":
        possible_option.append("Kiosk")
    if df.loc[i, "option_2"][11] == "1":
        possible_option.append("WI-FI")
    if df.loc[i, "option_2"][10] == "1":
        possible_option.append("Separate restrooms")
    if df.loc[i, "option_2"][9] == "1":
        possible_option.append("Pets")
    if df.loc[i, "option_2"][8] == "1":
        possible_option.append("Smart order")
    if df.loc[i, "option_2"][7] == "1":
        possible_option.append("Free parking")
    if df.loc[i, "option_2"][6] == "1":
        possible_option.append("Take out")
    if df.loc[i, "option_2"][5] == "1":
        possible_option.append("Possible to book")
    if df.loc[i, "option_2"][4] == "1":
        possible_option.append("Barrier-free")
    if df.loc[i, "option_2"][3] == "1":
        possible_option.append("Playroom")
    if df.loc[i, "option_2"][2] == "1":
        possible_option.append("Multilingual Menu")
    if df.loc[i, "option_2"][1] == "1":
        possible_option.append("Parcel service")
    if df.loc[i, "option_2"][0] == "1":
        possible_option.append("Mobile payment")
    result.append(", ".join(possible_option))
del df['option_2']
df["Facility"] = result

In [6]:
# 사용 컬럼 선택
col_s = ['id', 'name', 'telephone', 'address', 'address_road', 'holiday',
       'optime', 'subway', 'etc', 'description', 'table_chair',
       'floor_table_chair', 'room', 'product_price_min', 'product_price_max',
       'store_category_name', 'store_label_name', 'google_reservation', 'Facility']
col_p = ['product_name', 'product_id', 'is_package', 'is_main',
       'is_reserve', 'minimum_buy_count', 'price', 'product_description', 'includes']

#메뉴정보
for i in df[col_p].values :
    print("{"+' , '.join(list(map(lambda x,y : " : ".join([str(x),str(y)]), col_p, i.tolist())))+"}","\n,")

{product_name : [Reservation] Basque Cheese Cake Size 2(Reserve 3days in advance) , product_id : 40121 , is_package : 0 , is_main : 1 , is_reserve : 0 , minimum_buy_count : 1 , price : 46000.0 , product_description : *This product provides special packaging services only for the vacation shop customers. , includes : [Reservation] Basque Cheese Cake Size 2} 
,
{product_name : [Reservation] Macaron SET(Reserve 5days in advance) , product_id : 40124 , is_package : 0 , is_main : 1 , is_reserve : 0 , minimum_buy_count : 1 , price : 21000.0 , product_description : *This product provides special packaging services only for the vacation shop customers.
* Please note that the same type of macaron may be included or changed. , includes : [Reservation] Macaron SET(Reserve 5days in advance)
*Please note, the same type of macaron may be included or the types may change.} 
,
{product_name : [Reservation] Citrus Pound Cake(Reserve 3days in advance) , product_id : 41059 , is_package : 0 , is_main : 1 

## 모델

### 질문 리스트

In [7]:
q = """Please recommend tourist attractions nearby.
Is it possible to have a group dinner?
Is there another branch?
Is there a gender toilet?
How many seats are the stores in the store?
Have you ever aired on TV? I think I saw it ...
Have you received it?
Do you have an outdoor (terrace) seat?
Where is the restroom?
Three of us are here. What should we eat?
What is the best menu?
Does you are here cafe sell Zero Cola?
Do you sell alcohol here, too?
Is there any food for the baby?
Is there a drink that goes well with Croissant Sandwich?
What kind of drink is there?
A menu is spicy a lot?
B has allergies. What menu does not go in?
I'm a Muslim. Is there a menu that I can't eat?
What is the signature menu?
What advice do you have about how to eat or combination?
How long does it take to come out?
Is there an SNS event?
Is it possible to pay for installment?
Where should I pay?
Can I park?
Is there a dessert?
Can my pets be admitted?
Can I adjust the spiciness?
Is it possible to package?
Is it possible to add side dishes?
Do you have WiFi?
Is there a table for 10 people?
Can I make a reservation for this store on Google?
I'm a nearby office worker, can I make a reservation at a vacation shop?
Until what time is this store open?
When is the break time?
When is the holiday day?
How much does it take in Myeong-dong station?""".split("\n")

### Playground Text Completion 모델

In [16]:
organization = '?'
api_key = '키'

openai.api_key = os.getenv(api_key)

for i in q[:1] :
    emo_prompt = """Act as a restaurant Guide. I want you to act as a restaurant guide. I will write you my store and product information and you will suggest a answer to customers.
    In some cases, data may not exist to ask questions. but you should anwser the question as long as you can, very kindly and not like AI
    data : 
    [store_id : 10083 , name : YOU ARE HERE CAFE , telephone : 02-771-2288 , address : 195-3, Euljiro 2(i)-ga, Jung-gu, Seoul , address_road : 23, Myeongdong 9-gil, Jung-gu, Seoul , holiday : Open all year round , optime : Everyday 08:00~23:00 , subway : Jonggak Station,Euljiro 1(il)ga Station,Euljiro 3(sam)ga Station,Myeongdong Station , etc : *Vouchers cannot be used from December 24th-December 25th.*Please fill out the ""Requests"" box to clarify whether the food is for here or to-go.*Reservation can be changed up to 2 days prior to the visit.*Please note that the product is prepared in advance. Therefore if you don\'t visit on the date of the reservation, it will be treated as a NO-SHOW and the payment cannot be refunded.*Waiting time may occur during crowded hours, as seats are not assigned at this store. (This is only applied for the Croissant Sandwich SET Menu.) , description : YOU ARE HERE CAFE is a bakery cafe where you can relax in a busy city.Daily bakeries and desserts are rich in flavor using natural fermented butter.We hope everyone who visits YOU ARE HERE CAFE will have a relaxing and good time. , table_chair : 75 , floor_table_chair : 0 , room : 0 , product_price_min : 1400.0 , product_price_max : 100000.0 , store_category_name : Cafe , store_label_name : Cafe , google_reservation : Y , Facility : Self service, WI-FI, Separate restrooms, Smart order, Take out, Possible to book, Multilingual Menu, Mobile payment'
    , product_information : [[product_name : [Reservation] Basque Cheese Cake Size 2(Reserve 3days in advance) , product_id : 40121 , is_package : 0 , is_main : 1 , is_reserve : 0 , minimum_buy_count : 1 , price : 46000.0 , product_description : *This product provides special packaging services only for the vacation shop customers. , includes : [Reservation] Basque Cheese Cake Size 2] 
    ,[product_name : [Reservation] Macaron SET(Reserve 5days in advance) , product_id : 40124 , is_package : 0 , is_main : 1 , is_reserve : 0 , minimum_buy_count : 1 , price : 21000.0 , product_description : *This product provides special packaging services only for the vacation shop customers.
    * Please note that the same type of macaron may be included or changed. , includes : [Reservation] Macaron SET(Reserve 5days in advance)
    *Please note, the same type of macaron may be included or the types may change.] 
    ,[product_name : [Reservation] Citrus Pound Cake(Reserve 3days in advance) , product_id : 41059 , is_package : 0 , is_main : 1 , is_reserve : 0 , minimum_buy_count : 1 , price : 11500.0 , product_description : *This product provides special packaging services only for the vacation shop customers. , includes : Citrus Pound Cake] 
    ,[product_name : [Reservation] Chocolate Pound Cake(Reserve 3days in advance) , product_id : 41060 , is_package : 0 , is_main : 1 , is_reserve : 0 , minimum_buy_count : 1 , price : 12000.0 , product_description : *This product provides special packaging services only for the vacation shop customers. , includes : Chocolate Pound Cake] 
    ,[product_name : [Reservation] Cookies SET , product_id : 51330 , is_package : 0 , is_main : 1 , is_reserve : 0 , minimum_buy_count : 1 , price : 32600.0 , product_description : *This product provides special packaging services only for the vacation shop customers. , includes : Chocolate Cookie 2+Lotus Cookie 2+Mini Cookie 1] 
    ,[product_name : [Reservation] Assorted Cookies SET , product_id : 51333 , is_package : 0 , is_main : 1 , is_reserve : 0 , minimum_buy_count : 1 , price : 11500.0 , product_description : *This product provides special packaging services only for the vacation shop customers. , includes : Chocolate Cookie 1+Lotus Cookie 1] 
    ,[product_name : [Reservation] Strawberry Cream Cake(Reserve 3days in advance) , product_id : 2714661 , is_package : 0 , is_main : 1 , is_reserve : 0 , minimum_buy_count : 1 , price : 45000.0 , product_description : *This product provides special packaging services only for the vacation shop customers. , includes : ] 
    ,[product_name : [Reservation] Basque Cheese Cake Size 1(Reserve 3days in advance) , product_id : 3448122 , is_package : 0 , is_main : 0 , is_reserve : 0 , minimum_buy_count : 1 , price : 37000.0 , product_description : *This product provides special packaging services only for the vacation shop customers. , includes : [Reservation] Basque Cheese Cake Size 1] ]]

    Question :
    {}
    Answer : 
    A : 
    """.format(i)
    response = openai.Completion.create(
    model="text-davinci-003",
    prompt=emo_prompt,
    temperature=0.25,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    api_key = api_key
    )
    #print(emo_prompt.replace("\n",'')+response['choices'][0]['text'].replace("\n",''))
    print("Q : {}".format(i))
    print("A : {}\n".format(response['choices'][0]['text'].replace("\n",'').replace("\t",'').replace("  ",' ')))

Q : Please recommend tourist attractions nearby.
A :   Hi there! If you're looking for tourist attractions nearby, I recommend visiting the nearby Myeongdong Shopping Street, which is only a 5 minute walk away from YOU ARE HERE CAFE. There are many shops, restaurants, and attractions to explore. You can also visit the nearby Jonggak Station, Euljiro 1(il)ga Station, Euljiro 3(sam)ga Station, and Myeongdong Station, which are all within walking distance.



## ㄴ 결과